In [78]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score

Мы будем использовать данные, взятые с датчиков акселерометров и гироскопов смартфонов Samsung Galaxy S3. Телефоны носили в кармане добровольцы в возрасте от 19 до 49 лет. Смартфоны постоянно фиксировали значения ускорения и скорости по трём измерениям, а поведение людей записывали на видео, чтобы вручную отметить, какую физическую активность осуществлял человек в тот или иной момент.   
Данные содержат следующие признаки:
- различные показатели с акселерометра и гироскопа;
- метка активности (физическая активность человека в конкретный момент).

Как видите, есть ряд активностей, обозначенных цифрами. Эти метки означают следующее:

- 1 — ходьба;
- 2 — подъём;
- 3 — спуск;
- 4 — сидение;
- 5 — стояние;
- 6 — лежание.

Попробуем на основе данных с гироскопа и акселерометра разделить активности людей на некоторые схожие по своим характеристикам группы. В идеале наблюдения во время ходьбы должны попасть в один кластер, наблюдения во время подъёма по лестнице — в другой и т. д.

In [11]:
X_train = np.loadtxt("./data/train.txt")
y_train = np.loadtxt("./data/train_labels.txt")
 
X_test = np.loadtxt("./data/test.txt")
y_test = np.loadtxt("./data/test_labels.txt")

(10299, 561)

### Задание 4.1
Так как изначально данные были представлены для решения задачи классификации, то они находятся в разных файлах (обучающая и тестовая выборки в соотношении 70/30). Соедините признаки так, чтобы сначала шла обучающая выборка, а затем — тестовая, и отдельно соедините значения целевых переменных (разумеется, в том же порядке).

Какая размерность получилась у набора данных с признаками?

In [12]:
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))
X.shape

(10299, 561)

### Задание 4.2
Теперь найдите число различных активностей, то есть на сколько кластеров в идеале должны разделиться наблюдения.

In [14]:
len(np.unique(y))

6

### Задание 4.3
Далее необходимо отмасштабировать признаки. Будем использовать для этого алгоритм `StandardScaler`. Примените его ко всем значениям признаков и впишите в качестве ответа значение первого признака для первого объекта, предварительно округлив его до двух знаков после точки-разделителя.

In [22]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)
round(X_norm[0,0],2)

0.21

### Задание 4.4
Пора переходить к кластеризации. Для начала определите оптимальное количество кластеров, используя внутренние меры кластеризации. Используйте все известные вам коэффициенты, реализуемые в библиотеке sklearn: коэффициент силуэта, индекс Калински — Харабаса и индекс Дэвиса — Болдина. В качестве алгоритма возьмите `k-means++`, в качестве значения параметра `random_state` — число `42`.

Выведите оптимальное количество кластеров для каждой метрики, перебирая значения от 2 до 9 включительно. Также введите значение каждой метрики, округлённое до двух знаков после точки-разделителя.

In [30]:
%%time 
silhouettes = pd.Series([],dtype=float)
ch_scores = pd.Series([],dtype=float)
db_scores = pd.Series([],dtype=float)

for n_clusters in range(2,10):
    km = KMeans(random_state=42,n_clusters=n_clusters)
    km.fit(X_norm)
    
    silhouette = silhouette_score(X_norm,km.labels_)
    ch = calinski_harabasz_score(X_norm,km.labels_)
    db = davies_bouldin_score(X_norm,km.labels_)
    
    silhouettes = silhouettes.append(pd.Series(silhouette,index=[n_clusters]))    
    ch_scores = ch_scores.append(pd.Series(ch,index=[n_clusters]))
    db_scores = db_scores.append(pd.Series(db,index=[n_clusters]))    

Wall time: 55.8 s


#### Коэффициент силуэта

In [35]:
print(f'Количество кластеров: {silhouettes.idxmax()}')
print(f'Значение: {silhouettes.max():.2f}')

Количество кластеров: 2
Значение: 0.39


#### Индекс Калински — Харабаса

In [36]:
print(f'Количество кластеров: {ch_scores.idxmax()}')
print(f'Значение: {ch_scores.max():.2f}')

Количество кластеров: 2
Значение: 7880.81


#### Индекс Дэвиса — Болдина

In [34]:
print(f'Количество кластеров: {db_scores.idxmin()}')
print(f'Значение: {db_scores.min():.2f}')

Количество кластеров: 2
Значение: 1.07


### Задание 4.5
Теперь давайте оценим качество кластеризации с точки зрения внешних мер. Реализуйте кластеризацию с помощью алгоритма `k-means`. Пусть количество кластеров будет соответствовать количеству активностей. Параметр `random_state = 42`. В качестве ответов введите значения получившихся мер, предварительно округлив их до двух знаков после точки-разделителя.

In [40]:
km = KMeans(n_clusters=6,init='random',random_state=42)
km.fit(X_norm)

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300, n_clusters=6,
       n_init=10, n_jobs=None, precompute_distances='auto', random_state=42,
       tol=0.0001, verbose=0)

In [43]:
# normalized_mutual_info_score(y,km.labels_)
print(f'Однородность:                         {homogeneity_score(y,km.labels_):.2f}')
print(f'Полнота:                              {completeness_score(y,km.labels_):.2f}')
# v_measure_score(y,km.labels_)
print(f'ARI (скорректированный индекс Рэнда): {adjusted_rand_score(y,km.labels_):.2f}')

Однородность:                         0.54
Полнота:                              0.58
ARI (скорректированный индекс Рэнда): 0.42


### Задание 4.6
Выясните, к каким кластерам были преимущественно отнесены различные активности (т. е. в какой кластер попало большинство наблюдений с этой активностью).

> Заметьте, что кластеры могут повторяться.

In [65]:
convert = {1: 'ходьба',2: 'подъём',3: 'спуск',4: 'сидение',5: 'стояние', 6: 'лежание'}
activity = pd.Series(y).map(convert)
labels = pd.Series(km.labels_)

labels.groupby(activity).apply(lambda x: x.mode()[0]) + 1

лежание    0
подъём     2
сидение    1
спуск      4
стояние    1
ходьба     2
dtype: int64

### Задание 4.7
Теперь попробуйте реализовать алгоритм k-means для двух кластеров (для того числа активностей, которое является оптимальным с точки зрения внутренних мер) и снова посмотреть, как алгоритм разобьёт активности по кластерам.

In [75]:
km = KMeans(n_clusters=2,init='random',random_state=42)
km.fit(X_norm)

convert = {1: 'ходьба',2: 'подъём',3: 'спуск',4: 'сидение',5: 'стояние', 6: 'лежание'}
activity = pd.Series(y).map(convert)
labels = pd.Series(km.labels_)

labels.groupby(activity).apply(lambda x: x.mode()[0]) + 1

лежание    2
подъём     1
сидение    2
спуск      1
стояние    2
ходьба     1
dtype: int64

### Задание 4.8
Вычислите значение полноты для разбиения на два кластера алгоритмом k-means. Ответ округлите до двух знаков после точки-разделителя.

In [76]:
print(f'Полнота: {completeness_score(y,km.labels_):.2f}')

Полнота: 0.98


### Задание 4.10
Давайте сравним полученный результат с агломеративной иерархической кластеризацией. Реализуйте её также для двух кластеров и вычислите значение полноты.

1. Какой алгоритм показывает наилучшее качество, если судить по полноте?

In [82]:
ac = AgglomerativeClustering(n_clusters=2)
ac.fit(X_norm)

print(f'Полнота: {completeness_score(y,ac.labels_):.2f}')

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='ward', memory=None, n_clusters=2,
                        pooling_func='deprecated')

2. Какое значение полноты получилось для агломеративной кластеризации?

In [ ]:
print(f'Полнота: {completeness_score(y,ac.labels_):.2f}')

# ==========================================

In [47]:
ct = pd.crosstab(y, km.labels_)
ct.index = ['ходьба', 'подъём', 
             'спуск', 'сидение', 'стояние', 'лежание']
ct.columns = list(range(1,7))
ct

,1,2,3,4,5,6
ходьба,0,0,903,78,741,0
подъём,0,0,1242,5,295,2
спуск,0,0,321,196,889,0
сидение,91,1238,1,0,0,447
стояние,0,1346,0,0,0,560
лежание,1556,54,5,0,0,329
